In [5]:
import os
import sys
from PIL import Image
import numpy

def execute(filename, fpath, color_deficit='d'):
    modified_filename = "%s-%s-%s" % ('daltonize', color_deficit, filename)
    head, tail = os.path.split(fpath)

    modified_fpath = os.path.join(head, modified_filename)

    # Look if requested image is already available
    if os.path.isfile(modified_fpath):
        return (modified_filename, modified_fpath)

    # Get image data
    im = Image.open(fpath)
    if im.mode in ['1', 'L']: # Don't process black/white or grayscale images
        return (filename, fpath)
    im = im.copy() 
    im = im.convert('RGB') 
    RGB = numpy.asarray(im, dtype=float)

    
    # Transformation matrix for Tritanope (a blue/yellow deficit - very rare)
    lms2lmst = numpy.array([[1,0,0],[0,1,0],[-0.395913,0.801109,0]])

    # Colorspace transformation matrices
    rgb2lms = numpy.array([[0.38971, 0.68898, -0.07868],[-0.22981, 1.18340, 0.04641],[0, 0, 1]])
    lms2rgb = numpy.array([[1.9101968, -1.1121239, 0.2019080],[0.3709501, 0.6290543, 0],[0, 0, 1]])

    # Daltonize image correction matrix
    err2mod = numpy.array([[0,0,0],[0.7,1,0],[0.7,0,1]])

    # Get the requested image correction
    if color_deficit == 't':
        lms2lms_deficit = lms2lmst
    else:
        return (filename, fpath)
    
    # Transform to LMS space
    LMS = numpy.zeros_like(RGB)               
    for i in range(RGB.shape[0]):
        for j in range(RGB.shape[1]):
            rgb = RGB[i,j,:3]
            LMS[i,j,:3] = numpy.dot(rgb2lms, rgb)

    # Calculate image as seen by the color blind
    _LMS = numpy.zeros_like(RGB)  
    for i in range(RGB.shape[0]):
        for j in range(RGB.shape[1]):
            lms = LMS[i,j,:3]
            _LMS[i,j,:3] = numpy.dot(lms2lms_deficit, lms)

    _RGB = numpy.zeros_like(RGB) 
    for i in range(RGB.shape[0]):
        for j in range(RGB.shape[1]):
            _lms = _LMS[i,j,:3]
            _RGB[i,j,:3] = numpy.dot(lms2rgb, _lms)


    # Calculate error between images
    error = (RGB-_RGB)

    # Daltonize
    ERR = numpy.zeros_like(RGB) 
    for i in range(RGB.shape[0]):
        for j in range(RGB.shape[1]):
            err = error[i,j,:3]
            ERR[i,j,:3] = numpy.dot(err2mod, err)

    dtpn = ERR + RGB
    
    for i in range(RGB.shape[0]):
        for j in range(RGB.shape[1]):
            dtpn[i,j,0] = max(0, dtpn[i,j,0])
            dtpn[i,j,0] = min(255, dtpn[i,j,0])
            dtpn[i,j,1] = max(0, dtpn[i,j,1])
            dtpn[i,j,1] = min(255, dtpn[i,j,1])
            dtpn[i,j,2] = max(0, dtpn[i,j,2])
            dtpn[i,j,2] = min(255, dtpn[i,j,2])

    result = dtpn.astype('uint8')
    
    # Save daltonized image
    im_converted = Image.fromarray(result, mode='RGB')
    im_converted.save(modified_fpath)
    return (modified_filename, modified_fpath)

In [6]:
img_number = 1
for root, dirs, files in os.walk("imgs"):
    for name in files:
        #print("\n")
        print (os.path.join(root, name)) # will print path of files 
        path = os.path.join(root, name)
       
        fname = name
       
        #print ("Daltonizing in progress...")

        colorblindness = { 'd': 'Deuteranope',
                        'p': 'Protanope',
                        't': 'Tritanope',}

        
        for col_deficit in ['t']:
            modified_filename, modified_fpath = execute(fname, path, col_deficit)
            if modified_fpath == sys.argv[1]:
                print ("Error while processing image: PIL/NumPy missing and/or source file is a grayscale image.")
            else:
                print ("Image successfully daltonized")

        img_number +=1